In [1]:
import pandas as pd

In [2]:
weather_df=pd.read_csv('daegu-utf-8.csv', encoding='utf-8-sig')
print(weather_df.columns)
print(weather_df['날짜'].dtype) # 날짜 컬럼은 object 타입

Index(['날짜', '지점', '평균기온(℃)', '최저기온(℃)', '최고기온(℃)'], dtype='object')
object


In [5]:
weather_df.columns=['날짜', '지점', '평균기온', '최저기온', '최고기온']
print(weather_df.columns)

Index(['날짜', '지점', '평균기온', '최저기온', '최고기온'], dtype='object')


In [6]:
weather_df['날짜']=pd.to_datetime(weather_df['날짜'], format='%Y-%m-%d')
print(weather_df['날짜'].dtype)

datetime64[ns]


In [7]:
print(weather_df.head())
print(weather_df.shape)
num_rows=weather_df.shape[0]    # shape(row, col), shape[0]: row의 개수
num_missing=num_rows-weather_df.count() #count(): 정상값의 개수
print(num_missing)


          날짜     지점  평균기온  최저기온  최고기온
0 1907-01-31  143.0   NaN  -7.0   0.8
1 1907-02-01  143.0   NaN   NaN   NaN
2 1907-02-02  143.0   NaN   NaN   NaN
3 1907-02-03  143.0   NaN   NaN   NaN
4 1907-02-04  143.0   NaN   NaN   NaN
(42887, 5)
날짜        1
지점        1
평균기온    704
최저기온    630
최고기온    632
dtype: int64


 ### 결측치(NaN) 처리
    • dropna(axis):	결측치 제거
    – axis=0:	Nan이 포함된 행 제거(각 column에서 row방향으로 동작),
    – axis=1:	Nan이 포함된 열 제거(각 row에서 column방향으로 동작)
    • fillna(0):	결측치을 0으로 변경
    • fillna(method=‘ffill’):	이전 값으로 변경(forward	fill)
    • fillna(method=‘bfill’):	이후 값으로 변경(backward	fill)
    • interpolate():	결측치 양쪽의 값으로 중간값 계산

In [8]:
weather_df=weather_df.dropna(axis=0)
print(weather_df.count())
print(weather_df.head())

날짜      42180
지점      42180
평균기온    42180
최저기온    42180
최고기온    42180
dtype: int64
            날짜     지점  평균기온  최저기온  최고기온
701 1909-01-01  143.0  -4.1  -8.0   0.1
702 1909-01-02  143.0  -0.8  -6.7   6.1
703 1909-01-03  143.0   1.0  -2.4   3.6
704 1909-01-04  143.0   4.2   0.0   9.2
705 1909-01-05  143.0   3.4  -0.4   9.0


#### - 결측치를 제거한 최종 데이터를 csv파일로 저장
• index	=	False:	인덱스 항목 저장 안함

• encoding=‘utf-8-sig’ (euc_kr이 아닌 utf-8로 저장)

In [9]:
weather_df.to_csv('daegu-utf8-df.csv', index=False, mode='w', encoding='utf-8-sig')

#### 특정 연도와 달의 최고, 최저 기온 평균값 계산
• 해당 연도와 달의 DataFrame 가져오기

• datetime 객체 접근
    – dt.year,	dt.month,	dt.day

In [10]:
print('특정 연도와 달의 최고, 최저 기온 평균값 계산')

year_df= weather_df[weather_df['날짜'].dt.year==2023]
month_df=weather_df[weather_df['날짜'].dt.month==8]
print(month_df.head())

특정 연도와 달의 최고, 최저 기온 평균값 계산
            날짜     지점  평균기온  최저기온  최고기온
913 1909-08-01  143.0  25.2  22.9  28.3
914 1909-08-02  143.0  25.1  23.8  27.8
915 1909-08-03  143.0  25.6  24.4  29.1
916 1909-08-04  143.0  28.2  24.8  34.1
917 1909-08-05  143.0  27.0  23.2  32.8


In [11]:
max_temp_mean=round(month_df['최고기온'].mean(),1)
min_temp_mean=round(month_df['최저기온'].mean(),1)

print(f'2023년 8월 최저기온 평균: {min_temp_mean}, 최고기온 평균: {max_temp_mean}')

2023년 8월 최저기온 평균: 22.4, 최고기온 평균: 31.2


#### 1990년대와 2010년대 최고 기온 비교 #1
§ 1990년대 특정 월의 기온 및 2010년대 특정 월의 최고 기온 변화

• 사용 데이터:	daegu-utf8-df.csv 사용

• 결과를 두 개의 선 그래프(plot)로 비교

In [12]:
def draw_two_plots(title, x_data, max_temp_list1, label_y1, max_temp_list2, label_y2):

    plt.rcParams['axes.unicode_minus']=False
    plt.figure(figsize=(10,4))
    plt.plot(x_data, max_temp_list1, marker='s', markersize=6, color='b', label=label_y1)
    plt.plot(x_data, max_temp_list2, marker='s', markersize=6, color='r', label=label_y2)
    plt.xticks(x_data)  # 모든 xtick값을 출력함

    # plt.ylim(10,40)
    plt.title(title)
    plt.legend()
    plt.show()

def main():
    search_month=int(input('비교할 월을 입력하세요: ')) 

    weather_df=pd.read_csv('daegu-utf-8.csv', encoding='utf-8-sig')
    weather_df['날짜']=pd.to_datetime(weather_df['날짜'], format='%Y-%m-%d')

    first_decade_max_temp_list=[0]*10   # 10년동안 각 월의 평균 저장
    second_decade_max_temp_list=[0]*10

    first_decade=1990
    second_decade=2010

    for year in range(10):
        first_decade_df=weather_df[(weather_df['날짜'].dt.year==first_decade+year)& (weather_df['날짜'].dt.month==search_month)]
        first_decade_max_temp_list[year]=round(first_decade_df['최고기온'].mean(),1)
        second_decade_df=weather_df[(weather_df['날짜'].dt.year==second_decade+year)& (weather_df['날짜'].dt.month==search_month)]
        second_decade_max_temp_list[year]=round(second_decade_df['최고기온'].mean(),1)

        